In [32]:
import sys
sys.path.append("..")

In [33]:
import os
import torch

import numpy as np
import pandas as pd
import torch.nn as nn
import datetime as dt
import scipy.io as cio
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from pathlib import Path
from IPython import display
from torch.nn.utils import weight_norm
from matplotlib.animation import FuncAnimation
from torch.utils.data import Dataset, DataLoader, RandomSampler

from src.model_pinn import PINN
from src.process_data import ProcessDataBrusselas , ProcessDataColombia

In [34]:
# Configuración del dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [35]:
# Carga de las rutas
model_path = Path().cwd().parent / "models" / "PINN_cunboy_epchos_30_lamb_0.0001.pth"
# model_path = Path().cwd().parent / "models" / "PINN_brusselas_epchos_1000_lamb_2.0.pth"
data_path = Path().cwd().parent / "data" / "raw" / "em_cunboy_251212_251231_full.parquet"
# data_path = Path().cwd().parent / "data" / "raw" / "weather_data.mat"
gif_dir_path_bru = Path().cwd().parent / "gifs"
WS_val_idx = np.array([1, 2, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 19])

In [36]:
# Carga de los datos
# process_data = ProcessDataBrusselas(data_path)
process_data = ProcessDataColombia(data_path)
process_data.load_data()

kwargs = {
    # "R":0.15,
    "n_days":14,
    "interval":1,
    "WS_val_idx": np.array([1, 2, 3, 5, 7, 9, 10, 11, 13, 14, 15, 16, 19])}

process_data.process_data(**kwargs)
train_data, val_data, pinn_grid, params = process_data.return_data()

Iniciando carga de /home/alejandro/pinns/data/raw/em_cunboy_251212_251231_full.parquet
Documento cargado en 0:00:00.040773 s.
--- Coordenadas Cartesianas y Proyecciones ---
--- OK ---
--- Eliminar NaNs ---
--- OK ---
--- Selección de días y ordenamiento por coor ---
---> Se registran 30.0 días de registros
---> 14 selected days
---> Interval of 10.0 min
--- OK ---
--- Centrado y creación de la malla ---
---> L: 288166.93, W: 31.24, P0: 99749.97, Re: 651415802841 --
#	self.X_PINN.shape=(90, 2017) , self.Y_PINN.shape=(90, 2017)	#
#	np.nanmin(self.T_WS)=0.000e+00 , np.nanmax(self.T_WS)=1.311e+02	#
#	np.nanmin(self.P_WS)=-3.969e+01 , np.nanmax(self.P_WS)=1.532e+01	#
#	np.nanmin(self.U_WS)=-6.878e-01 , np.nanmax(self.U_WS)=6.881e-01	#
#	np.nanmin(self.V_WS)=-7.256e-01 , np.nanmax(self.V_WS)=7.233e-01	#
#	np.nanmin(self.X_WS)=-1.281e-01 , np.nanmax(self.X_WS)=1.281e-01	#
#	np.nanmin(self.Y_WS)=-4.833e-01 , np.nanmax(self.Y_WS)=4.833e-01	#
--- OK ---
--- Separar entre validación y entrenamien

In [37]:
# Carga del modelo
# 2. Instanciar el modelo (Debe tener la misma arquitectura que el entrenado)
model = PINN(input_dim=3, output_dim=3, hidden_neurons=1200).to(device)

# 3. Cargar los pesos (Checkpoint)
if torch.cuda.is_available():
    checkpoint = torch.load(model_path)
else:
    # Si entrenaste en GPU y evalúas en CPU, necesitas map_location
    checkpoint = torch.load(model_path, map_location=torch.device('cpu'))

# El archivo .pth que guardamos es un diccionario con varias claves ('model_state_dict', 'optimizer_state_dict', etc.)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Pesos cargados exitosamente desde: {model_path}")
print(f"Loss del checkpoint (entrenamiento): {checkpoint['loss']:.4f}")

Pesos cargados exitosamente desde: /home/alejandro/pinns/models/PINN_cunboy_epchos_30_lamb_0.0001.pth
Loss del checkpoint (entrenamiento): 3.0014


In [38]:
# 4. Poner el modelo en modo de evaluación
model.eval()

# Desactivamos el cálculo de gradientes para ahorrar memoria y cómputo
with torch.no_grad():
    t = torch.from_numpy(pinn_grid["T"].flatten()[:,None]).float().to(device)
    x = torch.from_numpy(pinn_grid["X"].flatten()[:,None]).float().to(device)
    y = torch.from_numpy(pinn_grid["Y"].flatten()[:,None]).float().to(device)
    
    
    # Predicción (Forward pass)
    outputs = model(t, x, y)
    
    # Separar variables predichas
    u_pred = outputs[:, 0:1]
    v_pred = outputs[:, 1:2]
    p_pred = outputs[:, 2:3]

In [39]:
u_pred

tensor([[0.0085],
        [0.0085],
        [0.0085],
        ...,
        [0.0085],
        [0.0085],
        [0.0085]], device='cuda:0')

In [8]:
t_ws = torch.from_numpy(val_data["T"].flatten()[:,None]).float()
x_ws = torch.from_numpy(val_data["X"].flatten()[:,None]).float()
y_ws = torch.from_numpy(val_data["Y"].flatten()[:,None]).float()
u_ws = torch.from_numpy(val_data["U"].flatten()[:,None]).float()
v_ws = torch.from_numpy(val_data["V"].flatten()[:,None]).float()
p_ws = torch.from_numpy(val_data["P"].flatten()[:,None]).float()

In [9]:
print("u_pred (min , max) :" , u_pred.min() , u_pred.max())
print("v_pred (min , max) :" , v_pred.min() , v_pred.max())
print("p_pred (min , max) :" , p_pred.min() , p_pred.max())

u_pred (min , max) : tensor(-0.6373, device='cuda:0') tensor(0.6157, device='cuda:0')
v_pred (min , max) : tensor(-0.5188, device='cuda:0') tensor(0.3056, device='cuda:0')
p_pred (min , max) : tensor(-3.6920, device='cuda:0') tensor(7.1402, device='cuda:0')


In [10]:
u_pinn = torch.concat([t, x, y, u_pred], axis=1)
v_pinn = torch.concat([t, x, y, v_pred], axis=1)
p_pinn = torch.concat([t, x, y, p_pred], axis=1)

u_station = torch.concat([t_ws, x_ws, y_ws, u_ws], axis=1)
v_station = torch.concat([t_ws, x_ws, y_ws, v_ws], axis=1)
p_station = torch.concat([t_ws, x_ws, y_ws, p_ws], axis=1)


In [11]:
# Ordenar los arreglos por el tiempo
def sort_tensor(tensor):
    index_sort = torch.argsort(tensor[:,0])
    return tensor[index_sort]

u_pinn = sort_tensor(u_pinn)#.reshape(2016, 160, -1)
v_pinn = sort_tensor(v_pinn)#.reshape(2016, 160, -1)
p_pinn = sort_tensor(p_pinn)#.reshape(2016, 160, -1)

u_station = sort_tensor(u_station)#.reshape(2016, 13, -1)
v_station = sort_tensor(v_station)#.reshape(2016, 13, -1)
p_station = sort_tensor(p_station)#.reshape(2016, 13, -1)

In [12]:
def generar_gif(n_pinn, n_station, label, ticks, ticklabels, title):
    
    
    # Generar malla para gráficos de nivel
    unique_t = torch.unique(n_pinn[:,0])
    unique_x = torch.unique(n_pinn[:,1])
    unique_y = torch.unique(n_pinn[:,2])
    
    x_temp = n_pinn[n_pinn[:,0] == 0, 1]
    y_temp = n_pinn[n_pinn[:,0] == 0, 2]
    
    x_temp = x_temp.reshape((unique_x.shape[0], -1))
    y_temp = y_temp.reshape((unique_x.shape[0], -1))
    
    # nmin , nmax
    nmin, nmax = torch.min(n_pinn[:,3]) , torch.max(n_pinn[:,3])
    
    kwargs_lv = {
    "vmin" : nmin,
    "vmax" : nmax,
    "levels" : np.linspace(nmin, nmax, 50),
    "cmap": "viridis"
    }
    
    kwargs_sc = {
    "vmin" : nmin,
    "vmax" : nmax,
    "cmap": "viridis",
    "edgecolors" : "black",
    "linewidths" : 0.5,
    "s" : 50
    }
    
    fig, ax = plt.subplots(figsize=(8,3))
    
    def draw_frame(t):
    
        ax.clear()
        
        # pinn
        _ , cantidad = torch.unique(n_pinn[:,0], return_counts=True)
        cantidad = torch.unique(cantidad)[0]
        
        n_pinn_temp = n_pinn[t*cantidad:(t + 1)*cantidad]
        n_temp = n_pinn_temp[:,3]
        n_temp = n_temp.reshape((unique_x.shape[0], -1))
        
        # Datos para el scatter plot de las wswd
        x_ws = n_station[t*13:(t + 1)*13, 1]
        y_ws = n_station[t*13:(t + 1)*13, 2]
        p_ws = n_station[t*13:(t + 1)*13, 3]
            
        # Contour relleno
        cs = ax.contourf(x_temp , y_temp, n_temp, **kwargs_lv)
        
        # Color bar
        if not hasattr(draw_frame, "cbar"):
            draw_frame.cbar = fig.colorbar(cs, ax=ax, label=label)
            draw_frame.cbar.set_ticks(ticks)
            draw_frame.cbar.set_ticklabels(ticklabels)
            draw_frame.cbar.ax.set_ylabel(label, rotation = -90, va = "bottom")
        
        # scatter plot
        plt.scatter(x_ws, y_ws, c=p_ws, **kwargs_sc)
        
            
        ax.set_title(f"{t}")
        ax.set_xlabel("Eje X")
        ax.set_ylabel("Eje Y")
        
        return []
    
    ani = animation.FuncAnimation(fig, draw_frame, frames=len(unique_t), interval=200, blit=False)

    # Guardar a GIF (requiere Pillow)
    ani.save(title, writer=animation.PillowWriter(fps=60), dpi=120)
    plt.close(fig)
    ...

In [13]:
def make_ticks(n_min:float, n_max:float):

    n_min, n_max = np.round(n_min, 1) , np.round(n_max, 1)
    ticks = np.linspace(n_min, n_max, 5)
    ticklabels = [f"{t:.1f}" for t in ticks]

    return ticks , ticklabels


In [14]:
ticks_u , ticklabels_u = make_ticks(u_pred.cpu().min(), u_pred.cpu().max())
ticks_v , ticklabels_v = make_ticks(v_pred.cpu().min(), v_pred.cpu().max())
ticks_p , ticklabels_p = make_ticks(p_pred.cpu().min(), p_pred.cpu().max())

In [15]:
generar_gif(u_pinn.cpu(), u_station.cpu(), "Velocidad U", ticks_u, ticklabels_u, gif_dir_path_bru / "velocidad_u.gif")
generar_gif(v_pinn.cpu(), v_station.cpu(), "Velocidad V", ticks_v, ticklabels_v, gif_dir_path_bru / "velocidad_v.gif")
generar_gif(p_pinn.cpu(), p_station.cpu(), "Velocidad P", ticks_p, ticklabels_p, gif_dir_path_bru / "velocidad_p.gif")